In [8]:
from scrapy.crawler import CrawlerProcess
from src.spiders.CoffeeCaveSpider import CoffeeCaveSpider

process = CrawlerProcess(settings={
    "FEEDS": {
        "scrapped_pages/output_coffe_cave.csv": {"format": "csv"},
    },
})
process.crawl(CoffeeCaveSpider)
process.start()

2024-11-24 18:10:39 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-11-24 18:10:39 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.10.0, Python 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform Windows-10-10.0.22631-SP0
2024-11-24 18:10:39 [scrapy.addons] INFO: Enabled addons:
[]
2024-11-24 18:10:39 [py.warnings] WARNING: C:\Users\oneks\PycharmProjects\PAD_Project\.venv\lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

Se

ReactorNotRestartable: 

In [1]:
import pandas as pd

coffee_cave_df = pd.read_csv(r"scrapped_pages/output_coffe_cave.csv", delimiter=",")
coffee_cave_df.head()

,name,price
0,Kawa mielona Brazylia Santos 1kg,83.00 zł
1,Kawa ziarnista Brazylia Santos 1kg,83.00 zł
2,Kawa mielona Brazylia Santos 500g,46.00 zł
3,Kawa ziarnista Brazylia Santos 500g,46.00 zł
4,Kawa mielona Brazylia Santos 250g,28.00 zł


In [2]:
coffee_cave_df.dropna(inplace=True)
print(coffee_cave_df)

                                    name      price
0       Kawa mielona Brazylia Santos 1kg   83.00 zł
1     Kawa ziarnista Brazylia Santos 1kg   83.00 zł
2      Kawa mielona Brazylia Santos 500g   46.00 zł
3    Kawa ziarnista Brazylia Santos 500g   46.00 zł
4      Kawa mielona Brazylia Santos 250g   28.00 zł
..                                   ...        ...
115    Kawa ziarnista Etiopia Sidamo 1kg  129.00 zł
116     Kawa mielona Etiopia Sidamo 500g   67.00 zł
117   Kawa ziarnista Etiopia Sidamo 500g   67.00 zł
118     Kawa mielona Etiopia Sidamo 250g   32.00 zł
119    Kawa ziarnista Etiopia Sidamo 250   32.00 zł

[120 rows x 2 columns]


In [3]:
import re
import numpy as np

def clean_price(column):
    cleaned_price = re.sub(r'[^\d,]', '', column)
    return cleaned_price[:-2] + "," + cleaned_price[-2:]

def extract_weight(product_name):
    match = re.search(r'(\d+([,.]\d+)?)\s?(kg|g)', product_name, re.IGNORECASE)
    if match:
        weight_value = match.group(1)
        weight_unit = match.group(3).lower()
        if weight_unit == 'g':
            weight_value = float(weight_value.replace(',', '.')) / 1000
        else:
            weight_value = float(weight_value.replace(',', '.'))

        clean_name = product_name.replace(match.group(0), '').strip()
        return weight_value, clean_name

    else:
        number_match = re.search(r'(\d+)$', product_name)
        if number_match:
            weight_value = int(number_match.group(1)) / 1000
            clean_name = product_name.replace(number_match.group(1), '').strip()
            return weight_value, clean_name
        else:
            return np.nan, product_name

coffee_cave_df["price"] = coffee_cave_df["price"].apply(clean_price)
coffee_cave_df["product_weight"], coffee_cave_df["name"] = zip(*coffee_cave_df["name"].apply(extract_weight))

coffee_cave_df.rename(columns={
    "price": "new_price [PLN]",
    "name": "product_name",
    "product_weight": "product_weight",
}, inplace=True)
coffee_cave_df["new_price [PLN]"] = coffee_cave_df["new_price [PLN]"].str.replace(",", ".").astype(float)

coffee_cave_df.head()

,product_name,new_price [PLN],product_weight
0,Kawa mielona Brazylia Santos,83.0,1.00
1,Kawa ziarnista Brazylia Santos,83.0,1.00
2,Kawa mielona Brazylia Santos,46.0,0.50
3,Kawa ziarnista Brazylia Santos,46.0,0.50
4,Kawa mielona Brazylia Santos,28.0,0.25


In [4]:
coffee_cave_df.to_csv("cleaned_data/coffee_cave_cleaned.csv", index=False, encoding='utf-8')